<a href="https://colab.research.google.com/github/nicolaiberk/_rrpviol_med/blob/master/_sc/BERT_estimates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import os
import csv
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import islice
from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [3]:
# get full set of news articles
if not os.path.isfile('newspapers/_bild_articles.csv'):
    os.system('wget -O articles.zip https://www.dropbox.com/sh/bbf0655w9931xbk/AADQNpkipxBENPk4Gp5j1UaDa?dl=0')
    os.system('unzip articles.zip -d newspapers')
    os.system('rm articles.zip')

In [4]:
model_name = 'bert-base-german-cased'
BERT_tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                               model_max_length = 256)

In [5]:
BERT_model = TFAutoModelForSequenceClassification.from_pretrained("drive/MyDrive/Bild/mig_clsfr_BERT")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at drive/MyDrive/Bild/mig_clsfr_BERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [6]:
BERT_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109081344 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  769       
Total params: 109,082,113
Trainable params: 109,082,113
Non-trainable params: 0
_________________________________________________________________


In [7]:
batch_size = 100

for paper in os.listdir('newspapers'):
  filename = 'newspapers/'+paper
  print(f'Processing file {paper}')
  with open(filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # define relative position in row based on title
        titlerow = np.argmax([r == 'title' for r in row])
        linkrow  = np.argmax([r == 'url'   for r in row])
        daterow  = np.argmax([r == 'date'  for r in row])
        textrow  = np.argmax([r == 'text'  for r in row])
        topicrow = np.argmax([r == 'topic' for r in row])
        break

    batch = []
    count = 0

    for row in tqdm(reader):
      if count >= batch_size:
          batch = pd.DataFrame(batch)
          input = BERT_tokenizer(list(batch[textrow]), padding='max_length', truncation=True, return_tensors='tf')
          batch['pred'] = np.array(BERT_model(input)['logits'][:,0])

          batch.to_csv('BERT_estimates.csv', mode='a', header=False)

          batch = []
          count = 0

      batch.append(row)
      count += 1
      
    if len(batch) > 0:
      batch.to_csv('BERT_estimates', mode='a', header=False)
  print(f'\tFinished file {paper}.')

    

0it [00:00, ?it/s]

Processing file _sz_articles.csv


5001it [00:46, 107.38it/s]

KeyboardInterrupt: ignored